In [1]:
# Imports
!pip install cpmpy numpy --quiet

import cpmpy as cp
import numpy as np
import time

def solve_with_time(model, time_limit=None):

    start_time = time.time()
    solution_found = model.solve(time_limit=time_limit)
    end_time = time.time()

    elapsed_time = end_time - start_time
    print("Time elapsed: {:.4f} seconds".format(elapsed_time))

    return solution_found

**Useful Resources:**
* CPMpy summary sheet: https://cpmpy.readthedocs.io/en/latest/summary.html
* CPMpy modeling documentation: https://cpmpy.readthedocs.io/en/latest/modeling.html

## **Session 4: Advanced Modelling**

### Outline:
1. Viewpoints
2. Channelling
3. Auxiliary Variables
4. Implied Constraints
5. Extra Exercises

## **1. Viewpoints**

In constraint programming, we often have multiple ways to represent the same problem. These representations, or "viewpoints," affect both how the CP solver explores the solution space and how easy it is to formulate the constraints. A carefully chosen viewpoint can:
- Reduce the complexity of the model, making it **faster** to solve
- Make the constraints **easier** to express
- **Implicitly** model constraints as part of the variables/domains definitions
- Improve **readability**

---

Consider the following problem:

We have $n$ pigeons and $m$ pigeonholes. Each pigeon must be assigned to one pigeonhole, but each pigeonhole can hold at most one pigeon.
Here, we consider $n=50$ pigeons and $m=60$ pigeonholes.

The output can be of the form:
```
Pigeon 1 goes to hole 1
Pigeon 2 goes to hole 7
...
Pigeon 50 goes to hole 53
```

Modelling this problem is quite straightforward. The focus is on constructing two models for this problem using **different viewpoints**.

### Viewpoint 1: Pigeons as variables

<details>
  <summary>Click to reveal hints</summary>

* We need a decision variable for each pigeon representing which hole it goes into.
* The [AllDifferent](https://cpmpy.readthedocs.io/en/latest/api/expressions/globalconstraints.html#cpmpy.expressions.globalconstraints.AllDifferent) constraint fits nicely when the pigeons are selected as decision variables.
</details>

In [2]:
num_pigeons = 50
num_pigeonholes = 60

# Decision variables
pigeons = cp.intvar(1, num_pigeonholes, shape=num_pigeons)  # Each pigeon is assigned to which pigeonhole?

# Constraints
model_vp1 = cp.Model()

# Each pigeonhole can hold at most one pigeon, thus no two pigeons in the same hole
model_vp1.add(cp.AllDifferent(pigeons))

# Solve the model
solved = solve_with_time(model_vp1)

if solved:
  print(model_vp1.status())
  for p in range(num_pigeons):
    print(f"Pigeon {p+1} goes to hole {pigeons[p].value()}")
else:
  print("No solution found.")

Time elapsed: 0.0866 seconds
ExitStatus.FEASIBLE (0.082447245 seconds)
Pigeon 1 goes to hole 50
Pigeon 2 goes to hole 49
Pigeon 3 goes to hole 48
Pigeon 4 goes to hole 47
Pigeon 5 goes to hole 46
Pigeon 6 goes to hole 45
Pigeon 7 goes to hole 44
Pigeon 8 goes to hole 43
Pigeon 9 goes to hole 42
Pigeon 10 goes to hole 41
Pigeon 11 goes to hole 40
Pigeon 12 goes to hole 39
Pigeon 13 goes to hole 38
Pigeon 14 goes to hole 37
Pigeon 15 goes to hole 36
Pigeon 16 goes to hole 35
Pigeon 17 goes to hole 34
Pigeon 18 goes to hole 33
Pigeon 19 goes to hole 32
Pigeon 20 goes to hole 31
Pigeon 21 goes to hole 30
Pigeon 22 goes to hole 29
Pigeon 23 goes to hole 28
Pigeon 24 goes to hole 27
Pigeon 25 goes to hole 26
Pigeon 26 goes to hole 25
Pigeon 27 goes to hole 24
Pigeon 28 goes to hole 23
Pigeon 29 goes to hole 22
Pigeon 30 goes to hole 21
Pigeon 31 goes to hole 20
Pigeon 32 goes to hole 19
Pigeon 33 goes to hole 18
Pigeon 34 goes to hole 17
Pigeon 35 goes to hole 16
Pigeon 36 goes to hole 15
Pi

### Viewpoint 2: Pigeonholes as variables

<details>
  <summary>Click to reveal hints</summary>

- Make sure that the domain includes also the option that the hole may be empty
- The constraint that each pigeon must be assigned to exactly one pigeonhole can be expressed in various ways, some of them are:
  - For each pigeon use `cp.sum` and ensure that it is equal to 1.
  - Use [GlobalCardinalityCount](https://cpmpy.readthedocs.io/en/latest/api/expressions/globalconstraints.html#cpmpy.expressions.globalconstraints.GlobalCardinalityCount)
  - Use [AllDifferentExcept0](https://cpmpy.readthedocs.io/en/latest/api/expressions/globalconstraints.html#cpmpy.expressions.globalconstraints.AllDifferentExcept0) and [Count](https://cpmpy.readthedocs.io/en/latest/api/expressions/globalfunctions.html#cpmpy.expressions.globalfunctions.Count)
</details>

In [3]:
# Decision variables
pigeonholes = cp.intvar(0, num_pigeons, shape=num_pigeonholes)  # Each pigeonhole holds which pigeon? 0 means that the hole is empty

# Constraints
model_vp2 = cp.Model()

# Each pigeon must be assigned to exactly one pigeonhole, there are various ways to model this constraint

# 1. Using the sum constraint
for i in range(1, num_pigeons + 1):
  model_vp2.add(cp.sum(pigeonholes == i) == 1)

# 2. Using the Global Cardinality Count constraint
model_vp2.add(cp.GlobalCardinalityCount(pigeonholes, list(range(1, num_pigeons + 1)), [1]*num_pigeons))

# 3. Using AllDifferentExcept0 and counting 0s
model_vp2.add(cp.AllDifferentExcept0(pigeonholes))  # a. Each number except 0 should be distinct
model_vp2.add(cp.Count(pigeonholes, 0) == num_pigeonholes - num_pigeons)  # b. All the pidgeons have to be assigned

# Solve the model
solved = solve_with_time(model_vp2)

if solved:
  holes_values = pigeonholes.value()
  for p in range(num_pigeons):
    for i, hole_value in enumerate(holes_values):
      if hole_value == p + 1:
        print(f"Pigeon {hole_value} goes to hole {i + 1}")

  print("The empty holes are:")
  for i, hole_value in enumerate(holes_values):
    if hole_value == 0:
      print(f"Hole {i + 1}")
else:
  print("No solution found.")

Time elapsed: 1.0468 seconds
Pigeon 1 goes to hole 17
Pigeon 2 goes to hole 5
Pigeon 3 goes to hole 49
Pigeon 4 goes to hole 4
Pigeon 5 goes to hole 13
Pigeon 6 goes to hole 37
Pigeon 7 goes to hole 25
Pigeon 8 goes to hole 20
Pigeon 9 goes to hole 15
Pigeon 10 goes to hole 44
Pigeon 11 goes to hole 39
Pigeon 12 goes to hole 60
Pigeon 13 goes to hole 18
Pigeon 14 goes to hole 48
Pigeon 15 goes to hole 22
Pigeon 16 goes to hole 40
Pigeon 17 goes to hole 1
Pigeon 18 goes to hole 30
Pigeon 19 goes to hole 55
Pigeon 20 goes to hole 46
Pigeon 21 goes to hole 42
Pigeon 22 goes to hole 24
Pigeon 23 goes to hole 53
Pigeon 24 goes to hole 7
Pigeon 25 goes to hole 3
Pigeon 26 goes to hole 29
Pigeon 27 goes to hole 50
Pigeon 28 goes to hole 19
Pigeon 29 goes to hole 16
Pigeon 30 goes to hole 31
Pigeon 31 goes to hole 58
Pigeon 32 goes to hole 2
Pigeon 33 goes to hole 26
Pigeon 34 goes to hole 12
Pigeon 35 goes to hole 35
Pigeon 36 goes to hole 23
Pigeon 37 goes to hole 8
Pigeon 38 goes to hole 41

**Observations:**
1. Which viewpoint was easier to model?
2. Which viewpoint was faster to solve?
3. Which constraints are implicitly covered in each viewpoint?

In [4]:
print("Viewpoint 1:")
time_vp1 = solve_with_time(model_vp1)
print("\nViewpoint 2:")
time_vp2 = solve_with_time(model_vp2)

Viewpoint 1:
Time elapsed: 0.0871 seconds

Viewpoint 2:
Time elapsed: 0.8439 seconds


## **2. Channeling**

Some constraints are more naturally expressed in one viewpoint, while other constraints of the same model are maybe more naturally expressed in another viewpoint. In such cases, **channeling** is a useful technique that connects variables from different viewpoints within a single model. Some key benefits:

1. You can use the most intuitive viewpoint for each constraint.
2. The solver may deduce information across linked viewpoints, potentially speeding up the solving process.
3. The final models can become easier to read, maintain, and extend.

Channeling is achieved by adding constraints that ensure consistency between the decision variables of different viewpoints.

---

Consider the *Student Seating Problem* from the lecture:

You are tasked with finding an optimal seating arrangement for `nStudents = 15` students across `nTables = 5` tables, where there are `nChairs = 20` chairs in total. You are given a list of chair assignments for each table:
`Chairs = [1..4, 5..8, 9..12, 13..16, 17..20]`.
The goal is to assign **all** students to the chairs in such a way that each table has either at least half or no chairs occupied.

Create variables with two different viewpoints and try to model each constraint with the viewpoint that is easier. Before solving, it is necessary that the variables between the viewpoints are logically connected.

<details>
  <summary>Click to reveal hints</summary>

There are two high-level constraints that need to be satisfied:
1. From a student perspective: **All** students sit on a **different** chair
2. From a chair perspective: In each table, either **all chairs are empty** or **at least half are non-empty**.
</details>

In [5]:
# Data
n_students = 15
n_chairs = 20
n_tables = 5

# Chair indexes for each table
# [1..4, 5..8, 9..12, 13..16, 17..20]
chairs_per_table = n_chairs // n_tables
tables = [list(range(start, start + chairs_per_table))
          for start in range(1, n_chairs + 1, chairs_per_table)]

# Model
model = cp.Model()

# Viewpoint 1: Students as vars
students = cp.intvar(1, n_chairs, shape=n_students)  # chair of each student

# Constraint: Each student seats at a different chair
# (easier with viewpoint 1)
model.add(cp.AllDifferent(students))

# Viewpoint 2: Chairs as vars
chairs = cp.intvar(0, n_students, shape=n_chairs)  # 0 means empty chair, otherwise the student on this chair

# Constraint: Each table must have either at least half or no chairs occupied
# (easier with viewpoint 2)
for table in tables:
  # which chair variables represent the current table
  subset_chairs = [chairs[c - 1] for c in table]

  all_empty = cp.sum(subset_chairs) == 0
  at_least_half_occupied = cp.Count(subset_chairs, 0) <= (len(table) // 2)
  model.add(all_empty | at_least_half_occupied)

# Channeling: At this point we have modeled half of the constraints with the
# vars of vp1 and half with those of vp2. We need to merge the two viewpoints.
# For each student, the chair of this student from vp1 has to point to this student in vp2
for i in range(n_students):
  chair_index = students[i] - 1
  model.add(chairs[chair_index] == i + 1)

# Solve the model
solved = solve_with_time(model)

if solved:
  print("Solution found:")
  print(f"Chair of each student: {students.value()}")

  for i, table in enumerate(tables):
    print(f"Table {i+1}: ", end="")
    subset_chairs = [chairs[c - 1] for c in table]
    for chair in subset_chairs:
      if chair.value() != 0:
        print(f"{chair.value()} ", end="")
    print()


Time elapsed: 0.0285 seconds
Solution found:
Chair of each student: [ 1  5  6 12 10 19  8 15 17  2 16 18  3 20  4]
Table 1: 1 10 13 15 
Table 2: 2 3 7 
Table 3: 5 4 
Table 4: 8 11 
Table 5: 9 12 6 14 


## **3. Auxiliary Variables**

Auxiliary variables can help in expressing complex constraints more naturally and clearly. They can be useful when establishing relationships between decision variables and additional properties or constraints, making it easier to formulate constraints. So, they are used:

* Because it is difficult to express the constraints in terms of the existing variables, or
* To allow the constraints to be expressed in a form that would propagate better.

**Car Sequencing**

Imagine an assembly line producing different types of cars, each with optional features (e.g. air-conditioning, sunroof etc.). Each station on the assembly line handles a specific feature and has a limited capacity, meaning it can only work on a certain percentage of the cars that pass through.

To prevent any station from being overwhelmed, cars must be arranged in a sequence so that the number of cars needing a particular feature never exceeds the station's capacity. For example, if a station can only handle half of the cars that pass by, then at most 1 out of every 2 cars in the sequence should require that feature.

A model of this problem is also available in the slides of lecture 5, page 23.
Try to model it without looking at the slides first.

In [6]:
# Data
at_most = [1, 2, 2, 2, 1]  # The maximum number of times an option can be present in a group of consecutive timeslots (see 'per_slots' below)
per_slots = [2, 3, 3, 5, 5]  # The number of consecutive timeslots (window) in which an option has a limit (see 'at_most' above)

demand = [1, 1, 2, 2, 2, 2]  # The demand per type of car
requires = cp.cpm_array([
    [1, 0, 1, 1, 0],
    [0, 0, 0, 1, 0],
    [0, 1, 0, 0, 1],
    [0, 1, 0, 1, 0],
    [1, 0, 1, 0, 0],
    [1, 1, 0, 0, 0]])  # Which options (columns) are needed for each car type (rows).

n_cars = sum(demand)  # The amount of cars to sequence
n_options = len(at_most)  # The amount of different options
n_types = len(demand)  # The amount of different car types

# Decision Variables
sequence = cp.intvar(0, n_types - 1, shape=n_cars)  # The sequence of cars
option = cp.boolvar(shape=(n_cars, n_options))  # Sequence of different options based on the car type

# Model
car_seq_model = cp.Model()

# The amount of each type of car in the sequence has to be equal to the demand for that type
car_seq_model.add(cp.GlobalCardinalityCount(sequence, list(range(n_types)), demand))

# Make sure that the options in the table correspond to those of the car type
for s in range(n_cars):
    car_seq_model.add([option[s, o] == requires[sequence[s], o] for o in range(n_options)])

# Check that no more than "at most" car options are used per "per_slots" slots
for o in range(n_options):
    for s in range(n_cars - per_slots[o] + 1):
        slot_range = range(s, s + per_slots[o])
        car_seq_model.add((cp.sum(option[slot_range, o]) <= at_most[o]))

solve_with_time(car_seq_model)

print("Sequence:", end="  ")
for s in sequence.value():
    print(s, end=" ")
print()
print('------------------------------')

for idx, row in enumerate(option.value().T):
    row_int = [int(x) for x in row]
    print(f"Option {idx + 1}: ", end=" ")
    for x in row_int:
        print(x, end=" ")
    print()

Time elapsed: 0.0205 seconds
Sequence:  0 2 4 3 5 1 5 2 4 3 
------------------------------
Option 1:  1 0 1 0 1 0 1 0 1 0 
Option 2:  0 1 0 1 1 0 1 1 0 1 
Option 3:  1 0 1 0 0 0 0 0 1 0 
Option 4:  1 0 0 1 0 1 0 0 0 1 
Option 5:  0 1 0 0 0 0 0 1 0 0 


For reference, here is a solution of this problem:

```
Sequence:  0 2 5 1 5 3 4 2 3 4
------------------------------
Option 1:  1 0 1 0 1 0 1 0 0 1
Option 2:  0 1 1 0 1 1 0 1 1 0
Option 3:  1 0 0 0 0 0 1 0 0 1
Option 4:  1 0 0 1 0 1 0 0 1 0
Option 5:  0 1 0 0 0 0 0 1 0 0
```

## **4. Implied Constraints**

As taken from the lecture slides:

Can we do something to make our model quicker?
During the search process, the solver will explore several infeasible parts of the search tree. Can we avoid that?
Although implied constraints are logically redundant, and do not change the set of solutions, they can be used to reduce the search effort of the solving process.

Consider the **Magic Series problem**, also seen at lecture 5, page 29:

The element at index i in I = 0..(n-1) is the number of occurrences of i, for an integer n. Create a model that finds a magic series for n = 100.

Print the time taken to solve this model.

In [7]:
n = 100

# Decision Variables
Magic = cp.intvar(0, n, shape=n)

model = cp.Model(cp.GlobalCardinalityCount(Magic, list(range(n)), Magic))

solve_with_time(model)

print(Magic.value())

Time elapsed: 4.1676 seconds
[96  2  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  1  0  0  0]


We can significantly improve solving time by adding implied constraints.

Add implied constraints (try it initially without looking at them in the slides), and compare the time taken to solve the new model with the above one.

In [8]:
# Implied constraints
model.add(n == cp.sum(Magic))
model.add(n == cp.sum([i * Magic[i] for i in range(n)]))

solve_with_time(model)

print(Magic.value())

Time elapsed: 0.8224 seconds
[96  2  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  1  0  0  0]


## **5. Extra Exercises**

### Auxiliary Variables: Orthogonal Latin Squares

Consider the [Orthogonal Latin Squares](https://en.wikipedia.org/wiki/Mutually_orthogonal_Latin_squares) problem:

A Latin square is an $ n \times n $ array filled with $n$ different symbols, each occurring exactly once in each row and exactly once in each column. In other words, a Latin Square is a square grid where:

* Each row contains all $n$ distinct symbols exactly once.
* Each column contains all $n$ distinct symbols exactly once.
* No symbol appears twice in the same row or column.

Two Latin squares are said to be orthogonal if, when placed on top of each other, each possible combination of their symbols appears exactly once.

**Instructions:**

You have a set of $ n $ integers (0..n-1), and your task is to construct two $ n \times n $ Latin squares that are orthogonal.

1. Define decision variables for the Latin squares.
2. Introduce auxiliary variables to represent each cell's pairing from both Latin squares.
3. Ensure that each pairing is unique across the entire grid.

In [9]:
n = 7  # Size of the Latin square, and number of symbols

# Decision variables for two Latin squares
square1 = cp.intvar(0, n-1, shape=(n, n), name="square1")
square2 = cp.intvar(0, n-1, shape=(n, n), name="square2")

# Auxiliary variables to combine symbols from both squares
pairing = cp.intvar(0, n*n-1, shape=(n, n), name="pairing")

# Model
model = cp.Model()

# Latin square constraints: all rows and columns should be all different for both squares
for i in range(n):
    model.add([cp.AllDifferent(square1[i, :]), cp.AllDifferent(square1[:, i])])
    model.add([cp.AllDifferent(square2[i, :]), cp.AllDifferent(square2[:, i])])

# Define the pairings
for i in range(n):
    for j in range(n):
        model.add(pairing[i, j] == n * square1[i, j] + square2[i, j])

# All pairings should be unique
model.add(cp.AllDifferent(pairing.flatten()))

# Solve the model
solved = solve_with_time(model)

if solved:
    print("Solution found:")
    print("Square 1:")
    print(square1.value())
    print("Square 2:")
    print(square2.value())
    print("Pairings:")
    print(pairing.value())
else:
    print("No solution found.")

Time elapsed: 0.9241 seconds
Solution found:
Square 1:
[[0 6 5 1 2 4 3]
 [5 1 3 4 6 0 2]
 [3 5 2 0 1 6 4]
 [4 0 6 5 3 2 1]
 [6 3 4 2 0 1 5]
 [2 4 1 6 5 3 0]
 [1 2 0 3 4 5 6]]
Square 2:
[[0 3 5 4 6 2 1]
 [3 1 6 0 2 4 5]
 [5 4 2 6 0 1 3]
 [1 5 0 2 4 3 6]
 [6 2 4 1 3 5 0]
 [4 6 3 5 1 0 2]
 [2 0 1 3 5 6 4]]
Pairings:
[[ 0 45 40 11 20 30 22]
 [38  8 27 28 44  4 19]
 [26 39 16  6  7 43 31]
 [29  5 42 37 25 17 13]
 [48 23 32 15  3 12 35]
 [18 34 10 47 36 21  2]
 [ 9 14  1 24 33 41 46]]


Now, to see the benefit of auxiliary variables, try to model the same problem but without using `pairings`. The orthogonality constraint needs to be formulated directly over the square variables. This is more complicated to model, and possibly also slower for the solver (check the run time).

In [10]:
n = 7  # Size of the Latin square, and number of symbols

# Decision variables for two Latin squares
square1 = cp.intvar(0, n-1, shape=(n, n), name="square1")
square2 = cp.intvar(0, n-1, shape=(n, n), name="square2")

# Model
model_no_aux = cp.Model()

# Latin square constraints: all rows and columns should be all different for both squares
for i in range(n):
    model_no_aux.add([cp.AllDifferent(square1[i, :]), cp.AllDifferent(square1[:, i])])
    model_no_aux.add([cp.AllDifferent(square2[i, :]), cp.AllDifferent(square2[:, i])])

# Orthogonality constraint: Ensure each combination is unique by pair-wise comparing all pairs
pairs = [(i, j) for i in range(n) for j in range(n)]
for idx1, (i1, j1) in enumerate(pairs):
    for idx2, (i2, j2) in enumerate(pairs):
        if idx1 < idx2:
            # Ensure that no two different positions result in the same pair (square1, square2)
            model_no_aux.add((square1[i1, j1] != square1[i2, j2]) | (square2[i1, j1] != square2[i2, j2]))

# Solve the model
solved = solve_with_time(model_no_aux, time_limit=30)  # limit to 30 seconds

if solved:
    print("Solution found:")
    print("Square 1:")
    print(square1.value())
    print("Square 2:")
    print(square2.value())
else:
    print("No solution found.")

Time elapsed: 3.4399 seconds
Solution found:
Square 1:
[[6 3 4 2 1 0 5]
 [2 1 0 4 5 3 6]
 [3 2 5 1 4 6 0]
 [5 0 2 6 3 4 1]
 [4 5 3 0 6 1 2]
 [0 6 1 3 2 5 4]
 [1 4 6 5 0 2 3]]
Square 2:
[[5 0 3 6 1 2 4]
 [2 5 1 4 0 3 6]
 [4 1 6 3 2 0 5]
 [3 4 0 1 6 5 2]
 [1 2 5 0 4 6 3]
 [6 3 4 2 5 1 0]
 [0 6 2 5 3 4 1]]


### Implied Constraints: Car Sequencing

Consider again the Car Sequencing problem. Add the implied constraints that are recommended in the slides of lecture 5, page 27.

For more details on the implied constraints of Car Sequencing, see this [article](https://www.researchgate.net/publication/220838036_Solving_the_Car_Sequencing_Problem_in_Constraint_Logic_Programming#fullTextFileContent), Pages 8-9.

In [11]:
# Implied constraints: Ensure we don’t stay too far below capacity
for o in range(n_options):
  # Total number of cars requiring this option
  M = sum([demand[car_type] * requires[car_type, o] for car_type in range(n_types)])

  # Total number of cars
  N = sum(demand)

  M_i = at_most[o]
  N_i = per_slots[o]

  k=0
  while (M - k*M_i) >= 0:
    car_seq_model.add(cp.sum(option[:N - k * N_i, o]) >= M - k * M_i)
    k += 1

solve_with_time(car_seq_model)

print("Sequence:", end="  ")
for s in sequence.value():
    print(s, end=" ")
print()
print('------------------------------')

for idx, row in enumerate(option.value().T):
    row_int = [int(x) for x in row]
    print(f"Option {idx + 1}: ", end=" ")
    for x in row_int:
        print(x, end=" ")
    print()

Time elapsed: 0.0293 seconds
Sequence:  0 1 5 2 4 3 3 4 2 5 
------------------------------
Option 1:  1 0 1 0 1 0 0 1 0 1 
Option 2:  0 0 1 1 0 1 1 0 1 1 
Option 3:  1 0 0 0 1 0 0 1 0 0 
Option 4:  1 1 0 0 0 1 1 0 0 0 
Option 5:  0 0 0 1 0 0 0 0 1 0 
